In [1]:
from preprocess import preprocess_data_flat
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_validate
from sklearn.metrics import f1_score, make_scorer, accuracy_score, balanced_accuracy_score, roc_auc_score
from xgboost import XGBClassifier
from tabulate import tabulate 

In [2]:
df_final = [preprocess_data_flat(x, 30) for x in ["nico", "sofia"]]
x_final, y_final = [pd.concat([x[i] for x in df_final], ignore_index=True) for i in range(2)]
X_train_flat, X_test_flat, y_train_flat, y_test_flat = train_test_split(
    x_final.to_numpy(), y_final.to_numpy(), test_size=0.2, random_state=123
)

In [3]:
model = XGBClassifier(
    random_state=123,
    eval_metric="logloss",
)

param_dist = {
    "max_depth": [2, 3, 4],
    "learning_rate": [0.01, 0.05, 0.1],
    "n_estimators": [100, 200, 300, 400],
    "min_child_weight": [3, 5, 7, 10],
    "subsample": [0.6, 0.7, 0.8],
    "colsample_bytree": [0.6, 0.7, 0.8],
    "reg_alpha": [0.5, 1.0, 2.0, 5.0],
    "reg_lambda": [1.0, 2.0, 5.0, 10.0],
    "gamma": [0.1, 0.5, 1.0, 2.0],
    "colsample_bylevel": [0.6, 0.7, 0.8],
    "scale_pos_weight": [1, 3, 5, 10],
}

n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)

random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    scoring="balanced_accuracy",
    cv=cv,
    random_state=123,
    n_jobs=10,
    refit="balanced_accuracy",
    verbose=0,
    n_iter=100,
)

random_search.fit(X_train_flat, y_train_flat)
best_model = random_search.best_estimator_

In [4]:
scoring = {
    "accuracy": make_scorer(accuracy_score),
    "balanced_accuracy": make_scorer(balanced_accuracy_score),
    "roc_auc": make_scorer(roc_auc_score),
    "f1": make_scorer(f1_score),
}


cv_results = cross_validate(
    best_model,
    X_train_flat,
    y_train_flat,
    cv=cv,
    scoring=scoring,
    return_train_score=True,
    n_jobs=-1,
)


fold_results = []
for i in range(n_splits):
    fold_results.append(
        {
            "Fold": i + 1,
            "Train Acc": f"{cv_results['train_accuracy'][i]:.4f}",
            "Test Acc": f"{cv_results['test_accuracy'][i]:.4f}",
            "Train BA": f"{cv_results['train_balanced_accuracy'][i]:.4f}",
            "Test BA": f"{cv_results['test_balanced_accuracy'][i]:.4f}",
            "Train AUC": f"{cv_results['train_roc_auc'][i]:.4f}",
            "Test AUC": f"{cv_results['test_roc_auc'][i]:.4f}",
            "Train F1": f"{cv_results['train_f1'][i]:.4f}",
            "Test F1": f"{cv_results['test_f1'][i]:.4f}",
            "Time (s)": f"{cv_results['fit_time'][i] + cv_results['score_time'][i]:.3f}",
        }
    )

summary_results = {
    "F1 Score": f"{cv_results['test_f1'].mean():.4f} ± {cv_results['test_f1'].std():.4f}",
    "Accuracy": f"{cv_results['test_accuracy'].mean():.4f} ± {cv_results['test_accuracy'].std():.4f}",
    "Balanced Accuracy": f"{cv_results['test_balanced_accuracy'].mean():.4f} ± {cv_results['test_balanced_accuracy'].std():.4f}",
    "ROC AUC": f"{cv_results['test_roc_auc'].mean():.4f} ± {cv_results['test_roc_auc'].std():.4f}",
    "Total Time (s)": f"{(cv_results['fit_time'] + cv_results['score_time']).sum():.3f}",
}

print("\n" + "=" * 80)
print("CROSS-VALIDATION RESULTS")
print("=" * 80)

print("\n RESULTS BY FOLD:")
print(tabulate(fold_results, headers="keys", tablefmt="pretty"))

print("\n STATISTICS (Test Set within CV):")
for metric, value in summary_results.items():
    print(f"   • {metric:<20}: {value}")


print("\n" + "=" * 80)
print("FINAL EVALUATION ON HELD-OUT TEST SET")
print("=" * 80)
y_pred_test = best_model.predict(X_test_flat)
y_prob_test = best_model.predict_proba(X_test_flat)[:, 1]

test_f1 = f1_score(y_test_flat, y_pred_test)
test_accuracy = accuracy_score(y_test_flat, y_pred_test)
test_balanced_accuracy = balanced_accuracy_score(y_test_flat, y_pred_test)
test_roc_auc = roc_auc_score(y_test_flat, y_prob_test)

print(f"   • {'F1 Score':<20}: {test_f1:.4f}")
print(f"   • {'Accuracy':<20}: {test_accuracy:.4f}")
print(f"   • {'Balanced Accuracy':<20}: {test_balanced_accuracy:.4f}")
print(f"   • {'ROC AUC':<20}: {test_roc_auc:.4f}")
print("=" * 80)


CROSS-VALIDATION RESULTS

 RESULTS BY FOLD:
+------+-----------+----------+----------+---------+-----------+----------+----------+---------+----------+
| Fold | Train Acc | Test Acc | Train BA | Test BA | Train AUC | Test AUC | Train F1 | Test F1 | Time (s) |
+------+-----------+----------+----------+---------+-----------+----------+----------+---------+----------+
|  1   |  0.9865   |  0.8514  |  0.9819  | 0.8086  |  0.9819   |  0.8086  |  0.9900  | 0.8952  |  0.944   |
|  2   |  0.9899   |  0.8514  |  0.9842  | 0.8033  |  0.9842   |  0.8033  |  0.9926  | 0.8952  |  0.812   |
|  3   |  0.9932   |  0.8784  |  0.9922  | 0.8233  |  0.9922   |  0.8233  |  0.9950  | 0.9159  |  0.929   |
|  4   |  0.9899   |  0.9189  |  0.9842  | 0.8858  |  0.9842   |  0.8858  |  0.9926  | 0.9423  |  0.898   |
|  5   |  0.9899   |  0.8919  |  0.9842  | 0.8442  |  0.9842   |  0.8442  |  0.9926  | 0.9245  |  0.939   |
+------+-----------+----------+----------+---------+-----------+----------+----------+-----